# Pancake Pridiction v2

In [96]:
from web3 import Web3
from web3.middleware import geth_poa_middleware
import datetime as dt
import pandas as pd
import numpy as n
import math

pd.set_option('display.max_rows', None)

In [97]:
import os
import sys

current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)
sys.path.append(project_dir) if project_dir not in sys.path else None

import pancake
psp = pancake.Prediction()

### Config

In [98]:
dataset_reconds = 300

### Fetching Data

In [99]:
round_columns = ["epoch",
                 "startTimestamp",
                 "lockTimestamp",
                 "closeTimestamp",
                 "lockPrice",
                 "closePrice",
                 "lockOracleId",
                 "closeOracleId",
                 "totalAmount",
                 "bullAmount",
                 "bearAmount",
                 "rewardBaseCalAmount",
                 "rewardAmount",
                 "oracleCalled"]

def get_history(_psp, current_epoch, back_in_time=100):
    start_epoch_history = current_epoch - 2 - back_in_time

    df_history_round = pd.DataFrame(columns=round_columns)
    for i in range(start_epoch_history, current_epoch - 1):
        df_round = _psp.get_round(i)
        df_history_round = df_history_round.append(df_round)

    df_history_round = df_history_round.sort_values('epoch', ascending=False)
    df_history_round = df_history_round.reset_index(drop=True)
    
    return df_history_round

#### Creating a dataset

In [100]:
current_epoch = psp.get_current_epoch()
df = get_history(psp, current_epoch, dataset_reconds)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled
0,39797.0,1.643208e+09,1.643209e+09,1.643209e+09,396.140015,396.130005,0.0,0.0,30.915226,15.937403,14.977823,14.977823,29.987768,True
1,39796.0,1.643208e+09,1.643208e+09,1.643209e+09,395.070007,396.140015,0.0,0.0,21.509480,11.496294,10.013186,11.496294,20.864195,True
2,39795.0,1.643208e+09,1.643208e+09,1.643208e+09,395.339996,395.070007,0.0,0.0,37.587791,19.316414,18.271379,18.271379,36.460159,True
3,39794.0,1.643207e+09,1.643208e+09,1.643208e+09,394.250000,395.339996,0.0,0.0,46.984928,21.424862,25.560064,21.424862,45.575378,True
4,39793.0,1.643207e+09,1.643207e+09,1.643208e+09,397.070007,394.250000,0.0,0.0,39.408630,23.287849,16.120781,16.120781,38.226372,True


### Adding result

In [101]:
def add_result(lockPrice, closePrice):
    if lockPrice == closePrice:
        # draw
        return 0
    elif lockPrice > closePrice:
        # bear
        return -1
    elif lockPrice < closePrice:
        # bull
        return 1
        
df["result"] = df.apply(lambda row : add_result(row["lockPrice"], row["closePrice"]), axis=1)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled,result
0,39797.0,1.643208e+09,1.643209e+09,1.643209e+09,396.140015,396.130005,0.0,0.0,30.915226,15.937403,14.977823,14.977823,29.987768,True,-1
1,39796.0,1.643208e+09,1.643208e+09,1.643209e+09,395.070007,396.140015,0.0,0.0,21.509480,11.496294,10.013186,11.496294,20.864195,True,1
2,39795.0,1.643208e+09,1.643208e+09,1.643208e+09,395.339996,395.070007,0.0,0.0,37.587791,19.316414,18.271379,18.271379,36.460159,True,-1
3,39794.0,1.643207e+09,1.643208e+09,1.643208e+09,394.250000,395.339996,0.0,0.0,46.984928,21.424862,25.560064,21.424862,45.575378,True,1
4,39793.0,1.643207e+09,1.643207e+09,1.643208e+09,397.070007,394.250000,0.0,0.0,39.408630,23.287849,16.120781,16.120781,38.226372,True,-1


### Maximum Consequitive Occurance

In [102]:
countDf = pd.DataFrame({'result': list(df['result'][(df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(df['result'].groupby((df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

bear_count = countDf[countDf.result == -1]['runs'].max()
bull_count = countDf[countDf.result == 1]['runs'].max()
draw_count = countDf[countDf.result == 0]['runs'].max()

print(f"Bear {bear_count} / Bull {bull_count} / Draw {draw_count}")

Bear 10 / Bull 7 / Draw 1


## Strategies

In [103]:
running_columns = ["epoch", "position", "amount", "reward", "recent_loss", "factor"]
df_running = pd.DataFrame(columns=running_columns)

In [104]:
def bet(df_running, epoch, position, amount, recent_loss, factor):
    data = [epoch, position, amount, 0, recent_loss, factor]
    temp = pd.DataFrame(data=[data], columns=running_columns)
    df_running = df_running.append(temp)
    return df_running

    
def get_round_stats(df, epoch):
    df_round = df[df.epoch == epoch]

    total_amount = (df_round["bullAmount"] + df_round["bearAmount"]).iloc[0]
    if total_amount > 0:
        bull_ratio = ((df_round["bullAmount"] / total_amount) * 100).iloc[0]
        bear_ratio = ((df_round["bearAmount"] / total_amount) * 100).iloc[0]
        bear_pay_ratio = (total_amount / df_round["bearAmount"]).iloc[0]
        bull_pay_ratio = (total_amount / df_round["bullAmount"]).iloc[0]
    else:
        bull_ratio = None
        bear_ratio = None
        bear_pay_ratio = None
        bull_pay_ratio = None

    lockPrice = df_round["lockPrice"].iloc[0]
    closePrice = df_round["closePrice"].iloc[0]

    if lockPrice == closePrice:
        # draw
        result = 0
    elif lockPrice > closePrice:
        # bear
        result = -1
    elif lockPrice < closePrice:
        # bull
        result = 1
    
    return {"total_amount": total_amount,
           "bull_ratio": bull_ratio, "bear_ratio": bear_ratio,
           "bear_pay_ratio": bear_pay_ratio, "bull_pay_ratio": bull_pay_ratio,
           "result": result}


def update_reward(df, df_running, epoch, status):
    bet_value = df_running[df_running.epoch == epoch]["amount"].iloc[0]
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    
    if status == 0:
        df_running.loc[df_running.epoch == epoch, "reward"] = -1 * bet_value
        pay_ratio = 0
    elif status == 1:
        epoch_stats = get_round_stats(df, epoch)
        if bet_position == "bull":
            pay_ratio = epoch_stats["bull_pay_ratio"]
        elif bet_position == "bear":
            pay_ratio = epoch_stats["bear_pay_ratio"]

        df_running.loc[df_running.epoch == epoch, "reward"] = (bet_value * pay_ratio) - bet_value
    
    print(f"[{epoch}] - Factor {pay_ratio} - Bet {bet_value} - Position {position}")    
    return df_running


def check_result(df, df_running, epoch):
    
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    epoch_stats = get_round_stats(df, epoch)
    
    if bet_position == "bull" and epoch_stats["result"] == 1:
        result = 1
    elif bet_position == "bear" and epoch_stats["result"] == -1:
        result = 1
    else:
        result = 0
    
    df_running = update_reward(df, df_running, epoch, result)
    return df_running, epoch_stats["result"]
    

def overview(df_running):
    total_spent = df_running.sum()["amount"]
    total_loss = abs(df_running[df_running["reward"] < 0].sum()["reward"])
    loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    estimated_win = df_running[df_running["reward"] > 0].sum()["reward"]
    win_times = df_running[df_running["reward"] > 0].count()["reward"]
    estimated_gain = df_running.sum()["reward"]
    max_spent = df_running.max()["amount"]

    last_win_epoch = df_running[df_running["reward"] > 0].max()["epoch"]
    if last_win_epoch is None or math.isnan(last_win_epoch):
        recent_loss = abs(df_running.sum()["reward"])
        recent_loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    else:
        recent_loss = abs(df_running[df_running["epoch"] > last_win_epoch].sum()["reward"])
        recent_loss_times = df_running[(df_running["epoch"] > last_win_epoch)
                                       & (df_running["reward"] < 0)].count()["reward"]

    return {"total_spent": total_spent,
            "max_spent": max_spent,
            "total_loss": total_loss,
            "loss_times": loss_times,
            "estimated_win": estimated_win,
            "win_times": win_times,
            "estimated_gain": estimated_gain,
            "recent_loss": recent_loss,
            "recent_loss_times": recent_loss_times}


### Strategy 1: Trend

In [105]:
import statsmodels.api as sm

starting_epoch = df.min()["epoch"] + 20
maximum_epoch = df.max()["epoch"] - 1

# 0: Even / 1: Odd
strategy_remainder = 0
base_bet = 0.01
safe_bet = 0.2

current_epoch = starting_epoch
value = base_bet
print("Starting Epoch: ", starting_epoch)

Starting Epoch:  39517.0


In [106]:
def calculate_trend(df, current_epoch, back_in_time=6):
    df_history_round = df[(df.epoch < current_epoch) & (df.epoch >= current_epoch - back_in_time)]
    X = df_history_round.index
    X = X.astype(float)

    y = df_history_round['closePrice']
    y = y.astype(float)

    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    alpha = model.params['x1']
    return alpha

In [107]:
while current_epoch < maximum_epoch:
    if current_epoch % 2 == strategy_remainder:
        
        bet_status = overview(df_running)
        current_round_stats = get_round_stats(df, current_epoch)

        trend = calculate_trend(df, current_epoch, 3)
        print("Trend: ", trend)

        if trend < 0:
            # bull
            custom_factor = current_round_stats["bull_pay_ratio"] - safe_bet
            position = "bull"
        else:
            # bear
            custom_factor = current_round_stats["bear_pay_ratio"] - safe_bet
            position = "bear"

        value = (bet_status["recent_loss"] + base_bet) / (custom_factor - 1)
        if value < base_bet:
            value = base_bet

        df_running = bet(df_running, current_epoch, position, value, bet_status["recent_loss"], custom_factor)
        df_running, result = check_result(df, df_running, current_epoch)
        print(f"[{current_epoch}] - {position} - {value}")
    current_epoch += 1

Trend:  1.677291870117216
[39518.0] - Factor 1.851088047027588 - Bet 0.015358905827949073 - Position bear
[39518.0] - bear - 0.015358905827949073
Trend:  1.142837524414034
[39520.0] - Factor 2.580644130706787 - Bet 0.01 - Position bear
[39520.0] - bear - 0.01
Trend:  -1.1978454589844034
[39522.0] - Factor 0 - Bet 0.030290536980375114 - Position bull
[39522.0] - bull - 0.030290536980375114
Trend:  -0.5900115966796875
[39524.0] - Factor 1.6711843013763428 - Bet 0.08550908182357796 - Position bull
[39524.0] - bull - 0.08550908182357796
Trend:  0.04150390624997158
[39526.0] - Factor 1.2789463996887207 - Bet 0.1266682209629469 - Position bear
[39526.0] - bear - 0.1266682209629469
Trend:  -0.7514953613280966
[39528.0] - Factor 0 - Bet 0.012919642322493777 - Position bull
[39528.0] - bull - 0.012919642322493777
Trend:  0.5849914550780966
[39530.0] - Factor 1.4559829235076904 - Bet 0.08953582531377413 - Position bear
[39530.0] - bear - 0.08953582531377413
Trend:  -0.18992614746102277
[39532.0]

### Strategy 2: Same-Before

### Strategy 3: Random

### Overview

In [108]:
sum_reward = 0
df_running = df_running.reset_index(drop=True)

for index, row in df_running.iterrows():
        reward = row["reward"]
        sum_reward += reward
        
        df_running.loc[df_running.index == index, "acc_reward"] = sum_reward
        
df_running

,epoch,position,amount,reward,recent_loss,factor,acc_reward
0,39518.0,bear,0.015359,0.013072,0.000000,1.651088,0.013072
1,39520.0,bear,0.010000,0.015806,0.000000,2.380644,0.028878
2,39522.0,bull,0.030291,-0.030291,0.000000,1.330136,-0.001412
3,39524.0,bull,0.085509,0.057392,0.030291,1.471184,0.055980
4,39526.0,bear,0.126668,0.035334,0.000000,1.078946,0.091314
5,39528.0,bull,0.012920,-0.01292,0.000000,1.774015,0.078394
6,39530.0,bear,0.089536,0.040827,0.012920,1.255983,0.119221
7,39532.0,bull,0.011673,0.012335,0.000000,1.856678,0.131555
8,39534.0,bull,0.010000,-0.01,0.000000,2.486924,0.121555
9,39536.0,bear,0.028579,0.025716,0.010000,1.699825,0.147271


In [109]:
summary = overview(df_running)
print(summary)

{'total_spent': 6.758078398092787, 'max_spent': 0.566075587285032, 'total_loss': 2.377311087121962, 'loss_times': 55, 'estimated_win': 4.05053087442351, 'win_times': 84, 'estimated_gain': 1.673219787301546, 'recent_loss': 0.07691827877445373, 'recent_loss_times': 2}


### Counting Consecutive Win/Loss

In [110]:
def add_result_binary(reward):
    if reward > 0:
        # bull
        return 1
    else:
        # bear
        return -1
        
temp_df = df_running.copy()

temp_df["result"] = temp_df.apply(lambda row : add_result_binary(row["reward"]), axis=1)

countDf = pd.DataFrame({'result': list(temp_df['result'][(temp_df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(temp_df['result'].groupby((temp_df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

loss_count = countDf[countDf.result == -1]['runs'].max()
win_count = countDf[countDf.result == 1]['runs'].max()

print(f"Loss {loss_count} / Win {win_count}")

Loss 4 / Win 10
